# PRUEBAS KNN

In [197]:
import numpy as np
import pandas as pd
import string  


import nltk
import re

import scipy
import matplotlib.pyplot as plt
#from pylab import reParams 
import urllib 
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


from sklearn import metrics

In [179]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [180]:
train_data['keyword'].fillna('no keyword', inplace = True) 
train_data['keyword'] = train_data['keyword'].str.replace('%20', ' ')
train_data['location'].fillna('no location', inplace = True)

In [181]:
#ELIMINAR ID --> OUTLIER ?

train_data['length'] = train_data['text'].str.len()
#train_data.drop(['id'],1,inplace = True)
train_data.head()

,id,keyword,location,text,target,length
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88


# Feature Enginieering

In [182]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

#FUNCIONES UTILES

def only_letters(tweet):
    tweet = re.sub(r'http\S*', '', tweet)
    tweet = re.sub(r'[^a-z\s]', '', tweet)
    return tweet

stop_words=set(stopwords.words("english"))

def filter_stopwords(tokenized_text):
    filtered_words=[]
    for w in tokenized_text:
        if w not in stop_words:
            filtered_words.append(w)
    return filtered_words

lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(tweet):
    lemmatized_words = []
    for word in tweet:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    return lemmatized_words

def transform_to_text(tweet_words):
    return " ".join(tweet_words)

#Cleaning text

train_data['clean_text'] = train_data['text'].str.lower()

train_data['clean_text'] = train_data['clean_text'].apply(only_letters)    

#Tokenización

train_data['clean_text'] = train_data['clean_text'].apply(word_tokenize)

#Remove stopwords

train_data['clean_text'] = train_data['clean_text'].apply(filter_stopwords) 

#Lemmatization                                                                       

train_data['clean_text'] = train_data['clean_text'].apply(lemmatize_tweet)  

train_data['clean_text'] = train_data['clean_text'].apply(transform_to_text)  

In [183]:
train_data.head()

,id,keyword,location,text,target,length,clean_text
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69,deed reason earthquake may allah forgive u
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38,forest fire near la ronge sask canada
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133,resident asked shelter place notified officer ...
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65,people receive wildfire evacuation order calif...
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88,got sent photo ruby alaska smoke wildfire pour...


# TD-IDF

In [184]:
from sklearn.feature_extraction.text import TfidfVectorizer

texts = train_data['clean_text']

tf_idf = TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range= (1,2))

features = tf_idf.fit_transform(texts)

In [185]:
feature_words = tf_idf.get_feature_names()
len(feature_words)

10911

In [186]:
df_tf_idf = pd.DataFrame(data = features.todense(), columns = tf_idf.get_feature_names())
df_tf_idf.shape


df_tf_idf["_target_"] = train_data.target
df_tf_idf.head()

,aa,ab,aba,aba woman,abandon,abandoned,abandoned aircraft,abbott,abbswinston,abbswinston zionist,...,zipper,zipper bag,zombie,zombie apocalypse,zone,zone coming,zone dont,zouma,zouma flattened,_target_
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [187]:
train_data.loc[:,["clean_text","target"]].head(9)


#Construimos los datos de entrenamiento y de test
X = df_tf_idf.drop("_target_", axis=1)
y = df_tf_idf["_target_"]

In [188]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


#usar k-fold 


# estandarizamos las features
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)


# Building and Training model 

In [189]:
classifier = neighbors.KNeighborsClassifier()

In [190]:
classifier.fit(X_train,y_train)
print(classifier)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')


In [192]:
accuracy = classifier.score(X_train,y_train)
print(accuracy)

0.7133004926108374


# Evaluaiting or model againts test set 

In [193]:
y_expected = y_test
y_pred = classifier.predict(X_test)
print(y_pred)

[0 0 0 ... 0 0 0]


In [194]:
print(metrics.classification_report(y_expected,y_pred))

              precision    recall  f1-score   support

           0       0.63      0.98      0.77       874
           1       0.90      0.24      0.38       649

    accuracy                           0.66      1523
   macro avg       0.76      0.61      0.57      1523
weighted avg       0.75      0.66      0.60      1523



In [199]:
train_data_new = pd.read_csv("train.csv")
test_data_new = pd.read_csv("test.csv")

# word_count
train_data_new['word_count'] = train_data_new['text'].apply(lambda x: len(str(x).split()))
test_data_new['word_count'] = test_data_new['text'].apply(lambda x: len(str(x).split()))

# unique_word_count
train_data_new['unique_word_count'] = train_data_new['text'].apply(lambda x: len(set(str(x).split())))
test_data_new['unique_word_count'] = test_data_new['text'].apply(lambda x: len(set(str(x).split())))

# stop_word_count
train_data_new['stop_word_count'] = train_data_new['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))
test_data_new['stop_word_count'] = test_data_new['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))

# url_count
train_data_new['url_count'] = train_data_new['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
test_data_new['url_count'] = test_data_new['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# mean_word_length
train_data_new['mean_word_length'] = train_data_new['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test_data_new['mean_word_length'] = test_data_new['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

# char_count
train_data_new['char_count'] = train_data_new['text'].apply(lambda x: len(str(x)))
test_data_new['char_count'] = test_data_new['text'].apply(lambda x: len(str(x)))

# punctuation_count
train_data_new['punctuation_count'] = train_data_new['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
test_data_new['punctuation_count'] = test_data_new['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# hashtag_count
train_data_new['hashtag_count'] = train_data_new['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
test_data_new['hashtag_count'] = test_data_new['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

# mention_count
train_data_new['mention_count'] = train_data_new['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
test_data_new['mention_count'] = test_data_new['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

In [200]:
train_data_new.head()

,id,keyword,location,text,target,word_count,unique_word_count,stop_word_count,url_count,mean_word_length,char_count,punctuation_count,hashtag_count,mention_count
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,13,13,6,0,4.384615,69,1,1,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,7,7,0,0,4.571429,38,1,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,22,20,11,0,5.090909,133,3,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,8,8,1,0,7.125000,65,2,1,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,16,15,7,0,4.500000,88,2,2,0


8